In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def unpack_pickle(obj_name):
    file_path = "{0}.pkl".format(obj_name)
    n_bytes = 2**31
    max_bytes = 2**31 - 1
    data = bytearray(n_bytes)
    
    ## read
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    loaded_data = pickle.loads(bytes_in)
    return loaded_data

In [3]:
X = unpack_pickle('X')

In [4]:
y = unpack_pickle('y')

In [5]:
X.shape, y.shape

((7730792, 73), (7730792,))

In [6]:
subm = pd.read_csv('../sample_submission.csv')

In [7]:
subm.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.5
1,00000176ac758d54827acd545b6315a5,0.5
2,0000019dcefc128c2d4387c1273dae1d,0.5
3,0000055553dc51b1295785415f1a224d,0.5
4,00000574cefffeca83ec8adf9285b2bf,0.5


In [8]:
test = unpack_pickle('test')

In [9]:
test.shape, subm.shape

((7853253, 73), (7853253, 2))

In [10]:
X.columns.dtype == test.columns.dtype

True

In [11]:
test.select_dtypes(include='category').columns.tolist()

['Processor',
 'OsPlatformSubRelease',
 'SkuEdition',
 'Census_PrimaryDiskTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSArchitecture',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'EngineVersion_1',
 'EngineVersion_2',
 'AppVersion_1',
 'AppVersion_2',
 'AvSigVersion_1',
 'Census_OSVersion_1']

In [12]:
dtypes = {}
for i in X.columns:
    dtypes[i] = str(X[i].dtype)

In [13]:
dtypes

{'AVProductStatesIdentifier': 'float32',
 'AVProductsInstalled': 'float16',
 'AppVersion': 'float64',
 'AppVersion_1': 'float64',
 'AppVersion_2': 'float64',
 'AppVersion_3': 'float64',
 'AvSigVersion': 'float64',
 'AvSigVersion_1': 'float64',
 'AvSigVersion_2': 'float64',
 'AvSigVersion_3': 'float64',
 'Census_ActivationChannel': 'float64',
 'Census_ChassisTypeName': 'float64',
 'Census_FirmwareManufacturerIdentifier': 'float16',
 'Census_FirmwareVersionIdentifier': 'float32',
 'Census_FlightRing': 'float64',
 'Census_GenuineStateName': 'float64',
 'Census_HasOpticalDiskDrive': 'int8',
 'Census_InternalBatteryNumberOfCharges': 'float32',
 'Census_InternalBatteryType': 'float64',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float16',
 'Census_InternalPrimaryDisplayResolutionHorizontal': 'float16',
 'Census_InternalPrimaryDisplayResolutionVertical': 'float16',
 'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16',
 'Census_IsSecureBootEnabled': 'int8',
 'Census_IsTouchEnabled'

In [14]:
for i in dtypes:
    test[i] = test[i].astype(dtypes[i])

In [15]:
test.select_dtypes(include='category').columns.tolist()

[]

In [16]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc
gc.enable()

In [29]:
params = {
    'boosting_type': 'gbdt',
    'class_weight': None,
    'colsample_bytree': 0.9,
    'learning_rate': 0.1,
    'max_depth': 40,
    'metric': 'auc',
    'min_child_samples': 20,
    'min_child_weight': 0.001,
    'min_split_gain': 0.0,
    'n_estimators': 4000,
    'n_jobs': 4,
    'num_leaves': 31, 
    'objective': 'binary',
    'random_state': None,
    'reg_alpha': 0.1,
    'reg_lambda': 5,
    'silent': False,
    'subsample': 0.7,
    'subsample_for_bin': 200000,
    'subsample_freq': 0
}

In [18]:
sub=subm['MachineIdentifier'].to_frame()
sub['HasDetections']=0

In [19]:
sub.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0
1,00000176ac758d54827acd545b6315a5,0
2,0000019dcefc128c2d4387c1273dae1d,0
3,0000055553dc51b1295785415f1a224d,0
4,00000574cefffeca83ec8adf9285b2bf,0


In [20]:
kfold = 2

In [21]:
X = X.values
y = y.values

In [22]:
test = test.values

In [23]:
gc.collect()

21

In [30]:
skf = StratifiedKFold(n_splits=kfold, random_state=1900)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    dset = lgb.Dataset(X_train, label=y_train)
    dset_eval = lgb.Dataset(X_eval, label=y_eval)
    lgb_model = lgb.train(params, dset, 1000, dset_eval,
                          verbose_eval=100, early_stopping_rounds=100)
    sub['HasDetections'] += lgb_model.predict(test, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
    
sub.to_csv('second_submit.csv', index=False, float_format='%.5f') 
gc.collect()
sub.head()

 lgb kfold: 1  of  2 : 


/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:661: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.716724
[200]	valid_0's auc: 0.722455
[300]	valid_0's auc: 0.725123
[400]	valid_0's auc: 0.72692
[500]	valid_0's auc: 0.728036
[600]	valid_0's auc: 0.729252
[700]	valid_0's auc: 0.729927
[800]	valid_0's auc: 0.730802
[900]	valid_0's auc: 0.731382
[1000]	valid_0's auc: 0.731938
[1100]	valid_0's auc: 0.732365
[1200]	valid_0's auc: 0.732874
[1300]	valid_0's auc: 0.733148
[1400]	valid_0's auc: 0.733479
[1500]	valid_0's auc: 0.733781
[1600]	valid_0's auc: 0.733983
[1700]	valid_0's auc: 0.734131
[1800]	valid_0's auc: 0.734276
[1900]	valid_0's auc: 0.734483
[2000]	valid_0's auc: 0.734634
[2100]	valid_0's auc: 0.734776
[2200]	valid_0's auc: 0.734865
[2300]	valid_0's auc: 0.734933
[2400]	valid_0's auc: 0.735029
[2500]	valid_0's auc: 0.735105
[2600]	valid_0's auc: 0.735212
[2700]	valid_0's auc: 0.735297
[2800]	valid_0's auc: 0.735355
[2900]	valid_0's auc: 0.73545
[3000]	valid_0's auc: 0.735513
[3100]	valid_0's a

/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:661: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.71673
[200]	valid_0's auc: 0.722637
[300]	valid_0's auc: 0.725345
[400]	valid_0's auc: 0.726819
[500]	valid_0's auc: 0.727998
[600]	valid_0's auc: 0.728971
[700]	valid_0's auc: 0.729867
[800]	valid_0's auc: 0.730499
[900]	valid_0's auc: 0.731232
[1000]	valid_0's auc: 0.731762
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.731762


,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.575615
1,00000176ac758d54827acd545b6315a5,0.671484
2,0000019dcefc128c2d4387c1273dae1d,0.577678
3,0000055553dc51b1295785415f1a224d,0.479722
4,00000574cefffeca83ec8adf9285b2bf,0.553883
